In [ ]:
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "True"
import sys

module_path = os.path.abspath(
    os.path.join(
        "/Users/arthurstrauss/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/Coding_projects/Quantum_Optimal_Control"
    )
)
if module_path not in sys.path:
    sys.path.append(module_path)

In [1]:
# Qiskit imports
from qiskit import pulse, transpile
from qiskit.transpiler import InstructionDurations
from qiskit.circuit import (
    ParameterVector,
    QuantumCircuit,
    QuantumRegister,
    Gate,
    CircuitInstruction,
)
from dynamicsbackend_estimator import DynamicsBackendEstimator

import numpy as np
import tqdm
import time
from typing import Union, Optional, List, Sequence
import jax

jax.config.update("jax_enable_x64", True)
# tell JAX we are using CPU
jax.config.update("jax_platform_name", "cpu")
# import Array and set default backend
from qiskit_dynamics.array import Array

Array.set_default_backend("jax")

/Users/arthurstrauss/opt/anaconda3/envs/rl_qoc/lib/python3.9/site-packages/qiskit_dynamics/dispatch/backends/jax.py:34: UserWarning: The functionality in the perturbation module of Qiskit Dynamics requires a JAX version <= 0.4.6, due to a bug in JAX versions > 0.4.6. For versions 0.4.4, 0.4.5, and 0.4.6, using the perturbation module functionality requires setting os.environ['JAX_JIT_PJIT_API_MERGE'] = '0' before importing JAX or Dynamics.
  warnings.warn(


# Circuit macros for environment

Below we define functions defining our way of parametrizing the custom gate we intend to calibrate. In the pulse gate scenario, we define two functions for clearly separating the abstraction layers that are intertwined here.
The function ```apply_custom_gate``` creates a custom parametrized ```Gate``` and appends it to a ```QuantumCircuit``` given in input. The parametrization of the gate is provided by the ```params: ParameterVector``` argument of the function and is used to build a custom calibration of the gate. This custom calibration is defined in the second function called ```custom_schedule```, which builds a custom parametrized pulse schedule that can be attached to the gate calibration (see this tutorial for more info: https://qiskit.org/documentation/tutorials/circuits_advanced/05_pulse_gates.html).

In [5]:
from qiskit.providers import Backend
from qiskit_experiments.calibration_management import Calibrations
from basis_gate_library import FixedFrequencyTransmon, EchoedCrossResonance
from copy import deepcopy
from helper_functions import determine_ecr_params


def custom_schedule(
    backend: Backend,
    physical_qubits: list,
    params: ParameterVector,
    keep_symmetry: bool = True,
):
    """
    Define parametrization of the pulse schedule characterizing the target gate.
    This function can be customized at will, however one shall recall to make sure that number of actions match the
    number of pulse parameters used within the function (throught the params argument).
        :param backend: IBM Backend on which schedule shall be added
        :param physical_qubits: Physical qubits on which custom gate is applied on
        :param params: Parameters of the Schedule/Custom gate
        :param keep_symmetry: Choose if the two parts of the ECR tone shall be jointly parametrized or not

        :return: Parametrized Schedule
    """
    # Load here all pulse parameters names that should be tuned during model-free calibration.
    # Here we focus on real time tunable pulse parameters (amp, angle, duration)
    pulse_features = ["amp", "angle", "tgt_amp", "tgt_angle"]

    # Uncomment lines below to include pulse duration as tunable parameter
    # pulse_features.append("duration")
    # duration_window = 0

    global n_actions
    assert n_actions == len(
        params
    ), f"Number of actions ({n_actions}) does not match length of ParameterVector {params.name} ({len(params)})"

    new_params, _, _ = determine_ecr_params(backend, physical_qubits)

    qubits = tuple(physical_qubits)

    if keep_symmetry:  # Maintain symmetry between the two GaussianSquare pulses
        for sched in ["cr45p", "cr45m"]:
            for i, feature in enumerate(pulse_features):
                if feature != "duration":
                    new_params[(feature, qubits, sched)] += params[i]
                else:
                    new_params[
                        (feature, qubits, sched)
                    ] += pulse.builder.seconds_to_samples(duration_window * params[i])
    else:
        num_features = len(pulse_features)
        for i, sched in enumerate(["cr45p", "cr45m"]):
            for j, feature in enumerate(pulse_features):
                if feature != "duration":
                    new_params[(feature, qubits, sched)] += params[i * num_features + j]
                else:
                    new_params[
                        (feature, qubits, sched)
                    ] += pulse.builder.seconds_to_samples(
                        duration_window * params[i * num_features + j]
                    )

    cals = Calibrations.from_backend(
        backend,
        [
            FixedFrequencyTransmon(["x", "sx"]),
            EchoedCrossResonance(["cr45p", "cr45m", "ecr"]),
        ],
        add_parameter_defaults=True,
    )

    # Retrieve schedule (for now, works only with ECRGate(), as no library yet available for CX)
    parametrized_schedule = cals.get_schedule(
        "ecr", physical_qubits, assign_params=new_params
    )
    return parametrized_schedule

ModuleNotFoundError: No module named 'basis_gate_library'

In [ ]:
# Pulse gate ansatz


def apply_custom_gate(
    qc: QuantumCircuit,
    params: Optional[ParameterVector] = None,
    tgt_register: Optional[QuantumRegister] = None,
):
    """
    Define ansatz circuit to be played on Quantum Computer. Should be parametrized with Qiskit ParameterVector
    This function is used to run the QuantumCircuit instance on a Runtime backend
    :param qc: Quantum Circuit instance to add the gate on
    :param params: Parameters of the custom Gate
    :param tgt_register: Quantum Register formed of target qubits
    :return:
    """
    global n_actions, backend, target

    if params is None:
        params = ParameterVector("theta", n_actions)
    if tgt_register is None:
        tgt_register = qc.qregs[0]

    # Choose below which target gate you'd like to calibrate
    parametrized_gate = Gate("custom_ecr", 2, params=params.params)
    gate, physical_qubits = target["gate"], target["register"]
    parametrized_schedule = custom_schedule(
        backend=backend, physical_qubits=physical_qubits, params=params
    )
    qc.add_calibration(parametrized_gate, physical_qubits, parametrized_schedule)
    qc.append(parametrized_gate, tgt_register)

## Generic information characterizing the quantum system

The algorithm is built upon Qiskit modules. To specify how to address our quantum system of interest, we therefore adopt the IBM approach to define a quantum backend, on which qubits are defined and can be accessed via control actions and measurements.

The cell below specifies:
- ```physical_qubits```: List of qubit indices which are specifically addressed by controls , namely the ones for which we intend to calibrate a gate upon or steer them in a specific quantum state. Note that this list could include less qubits than the total number of qubits, which can be useful when one wants to take into account crosstalk effects emerging from nearest-neigbor coupling.
- ```sampling_Paulis```: number of Pauli observables  to be sampled from the system: the algorithm relies on the ability to process measurement outcomes to estimate the expectation value of different Pauli operators. The more observables we provide for sampling, the more properties we are able to deduce with accuracy about the actual state that was created when applying our custom controls. For a single qubit, the possible Pauli operators are $\sigma_0=I$, $\sigma_x=X$, $\sigma_y=Y$, $\sigma_z=Z$. For a general multiqubit system, the Pauli observables are tensor products of those single qubit Pauli operators. The algorithm will automatically estimate which observables are the most relevant to sample based on the provided target. The probability distribution from which those observables are sampled is derived from the Direct Fidelity Estimation (equation 3, https://link.aps.org/doi/10.1103/PhysRevLett.106.230501) algorithm. 
- ```N_shots```: Indicates how many measurements shall be done for each provided circuit (that is a specific combination of an action vector and a Pauli observable to be sampled)
- ```n_actions```: Indicates the number of pulse/circuit parameters that characterize our parametrized quantum circuit. For our pulse level ansatz, this number will depend on the number of parameters we are willing to tune in the original two-qubit gate calibration. 
- ```estimator_options```: Options of the Qiskit Estimator primitive. The Estimator is the Qiskit module enabling an easy computation of Pauli expectation values. One can set options to make this process more reliable (typically by doing some error mitigation techniques in post-processing). Works only with Runtime Backend at the moment
- ```abstraction_level``` chosen to encode our quantum circuit. One can choose here to stick to the usual circuit model of quantum computing, by using the ```QuantumCircuit``` objects from Qiskit and therefore set the ```abstraction_level``` to ```"circuit"```. However, depending on the task at hand, one can also prefer to use a pulse description of all the operations in our circuit. This is possible for both real backends and simulators by using respectively Qiskit Runtime and Qiskit Dynamics. In this case, one should define the ansatz circuit above in a pulse level fashion, and the simulation done at the Hamiltonian level, and not only via statevector calculations. In this notebook we set the ```abstraction_level``` to ```"pulse"```. Another notebook at the gate level is available in the repo.

In [ ]:
physical_qubits = [0, 1]
n_actions = 4  # Cf number of parameters in custom_schedule function above
abstraction_level = "pulse"

## Setting up Quantum Backend

### Simulation backend initialization: Qiskit Dynamics
If you want to run the algorithm over a simulation, one can use Qiskit Dynamics for pulse level simulation of quantum circuits. Below we set the ground for declaring a ```DynamicsBackend```.

This can be done in two ways: 

1. Declare a ```DynamicsBackend``` from a ```FakeBackend``` or ```IBMBackend``` instance and use the ```from_backend()``` method to retrieve the Hamiltonian description of such backend.
2. Alternatively, you can define your own custom Hamiltonian/Linblad that should be used to simulate the multiqubit system of interest, and feed it to a ```Solver``` instance which can be used to declare the ```DynamicsBackend```.
For more information you can check Qiskit Dynamics documentation (https://qiskit.org/documentation/dynamics/apidocs/backend.html)


In [ ]:
from qiskit.providers.fake_provider import FakeHanoi, FakeJakarta, FakeJakartaV2
from helper_functions import get_control_channel_map
from qiskit_dynamics import DynamicsBackend, Solver
from helper_functions import get_solver_and_freq_from_backend

fake_backend = FakeJakarta()
fake_backend_v2 = FakeJakartaV2()
control_channel_map = get_control_channel_map(fake_backend, physical_qubits)
dt = fake_backend_v2.target.dt
print("Coupling Map: ", list(fake_backend_v2.coupling_map.get_edges()))

In [ ]:
dynamics_options = {
    "seed_simulator": None,  # "configuration": fake_backend.configuration(),
    "control_channel_map": control_channel_map,
    "solver_options": {"method": "jax_odeint", "atol": 1e-6, "rtol": 1e-8, "hmax": dt},
}
qubit_properties = fake_backend_v2.qubit_properties(physical_qubits)

In [ ]:
# Using the from_backend method

dynamics_backend = DynamicsBackend.from_backend(
    fake_backend, subsystem_list=physical_qubits, **dynamics_options
)
dynamics_backend.target.qubit_properties = qubit_properties

In [ ]:
print(qubit_properties)

In [ ]:
# Example of target gate
from qiskit.circuit.library.standard_gates import ECRGate, XGate

ECR_tgt = {"gate": ECRGate(), "register": physical_qubits}
target = ECR_tgt
print(target)

In [ ]:
from helper_functions import perform_standard_calibrations

# Adds baseline calibrations for X, SX, RZ, and ECR gate (the latter is not properly calibrated, it just adds a calibration but without any experiments attached to find the good one)
cals, results = perform_standard_calibrations(backend)

In [ ]:
from qiskit.circuit import Parameter
from qiskit.primitives import BackendEstimator

estimator = BackendEstimator(dynamics_backend)
qc = QuantumCircuit(2)
apply_custom_gate(qc)
qc.draw("mpl")

In [ ]:
from qiskit.quantum_info import SparsePauliOp

obs = SparsePauliOp.from_list([("XX", 1.0), ("YY", 1.0)])

In [ ]:
estimator.run([qc] * 300, [obs] * 300, np.random.uniform())

In [ ]:
estimator.preprocessed_circuits[0][0].draw("mpl")

In [ ]:
estimator.preprocessed_circuits[0][1][0].draw("mpl")

In [ ]:
estimator.transpiled_circuits[0].draw("mpl")

In [ ]:
from qiskit.compiler import schedule

qc2 = transpile(qc, backend)
schedule(qc2, backend)

In [ ]:
from qiskit.quantum_info import SparsePauliOp

obs = SparsePauliOp.from_list([("XX", 1.0), ("ZZ", 1), ("IZ", 1)])

In [ ]:
job = estimator.run(qc, obs, [3.14])

In [ ]:
with pulse.build() as sched:
    pulse.play(pulse.Drag(param, 0.1, 20, 0.0), pulse.DriveChannel(0))
    pulse.delay(40, pulse.DriveChannel(0))
    pulse.play(pulse.Drag(150, 0.1, 20, 0.0), pulse.DriveChannel(0))

In [ ]:
sched.draw()

In [ ]:
def concatenate2(**kwargs):
    return kwargs


def concatenate(**kwargs):
    print(kwargs)
    for key in kwargs:
        if key == "Yo":
            kwargs.pop(key)
            break
    print(concatenate2(**kwargs))
    return kwargs


concatenate(hey="Well", ho="Ay", Yo="YI")